In [1]:
import duckdb
import pandas as pd
import sqlalchemy
%load_ext sql

In [2]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [3]:
%sql duckdb:///../../data/raw/train_data.db

In [5]:
%%sql 
SELECT * 
FROM duckdb_tables()

,schema_name,schema_oid,table_name,table_oid,internal,temporary,has_primary_key,estimated_size,column_count,index_count,check_constraint_count,sql
0,main,1,train_table,769,False,False,False,5531451,190,0,0,"CREATE TABLE train_table(""customer_ID"" VARCHAR..."


In [6]:
%%sql
SELECT *
FROM train_table
limit(5);

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,...,None,None,None,0.002427,0.003706,0.003818,None,0.000569,0.000610,0.002674
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,...,None,None,None,0.003954,0.003167,0.005032,None,0.009576,0.005492,0.009217
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954180,0.091505,0.021655,1.009672,0.006815,0.123977,0.007598,0.009423,...,None,None,None,0.003269,0.007329,0.000427,None,0.003429,0.006986,0.002603
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.960384,0.002455,0.013683,1.002700,0.001373,0.117169,0.000685,0.005531,...,None,None,None,0.006117,0.004516,0.003200,None,0.008419,0.006527,0.009600
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.947248,0.002483,0.015193,1.000727,0.007605,0.117325,0.004653,0.009312,...,None,None,None,0.003671,0.004946,0.008889,None,0.001670,0.008126,0.009827


In [7]:
%%sql
SELECT count(distinct customer_ID) from train_table

,"count(DISTINCT ""customer_ID"")"
0,458913


In [9]:
%%sql
SELECT n_rows, count(customer_ID) as customers
FROM
(
SELECT customer_ID, COUNT(1) as n_rows
from train_table
group by 1
)
group by 1
order by 1;

,n_rows,customers
0,1,5120
1,2,6098
2,3,5778
3,4,4673
4,5,4671
5,6,5515
6,7,5198
7,8,6110
8,9,6411
9,10,6721


In [19]:
%%sql
dates_df << WITH dates as 
(
SELECT distinct customer_ID,
       S_2::date as row_date,
       (lag(S_2) over(partition by customer_ID 
                      order by S_2 
                      rows between unbounded preceding 
                      and unbounded following))::date as prev_row_date
from train_table
)
SELECT customer_ID,
       row_date,
       prev_row_date,
       (row_date - prev_row_date) as days_passed
FROM dates

Returning data to local variable dates_df


In [26]:
dates_df[dates_df.customer_ID=="013ea122c3fded5bb90c0455b0b6dded41658b58ca601ca9da00a6bac3dc85bc"].sort_values(by="row_date")

,customer_ID,row_date,prev_row_date,days_passed
3,013ea122c3fded5bb90c0455b0b6dded41658b58ca601c...,2017-03-21,None,NaN
2765779,013ea122c3fded5bb90c0455b0b6dded41658b58ca601c...,2017-04-06,2017-03-21,16.0
1383531,013ea122c3fded5bb90c0455b0b6dded41658b58ca601c...,2017-05-23,2017-04-06,47.0
1383532,013ea122c3fded5bb90c0455b0b6dded41658b58ca601c...,2017-06-14,2017-05-23,22.0
4,013ea122c3fded5bb90c0455b0b6dded41658b58ca601c...,2017-07-21,2017-06-14,37.0
1383533,013ea122c3fded5bb90c0455b0b6dded41658b58ca601c...,2017-08-07,2017-07-21,17.0
1383534,013ea122c3fded5bb90c0455b0b6dded41658b58ca601c...,2017-09-13,2017-08-07,37.0
4149018,013ea122c3fded5bb90c0455b0b6dded41658b58ca601c...,2017-10-21,2017-09-13,38.0
2765780,013ea122c3fded5bb90c0455b0b6dded41658b58ca601c...,2017-11-20,2017-10-21,30.0
2765781,013ea122c3fded5bb90c0455b0b6dded41658b58ca601c...,2017-12-21,2017-11-20,31.0


In [24]:
dates_df.customer_ID.iloc[3]

'013ea122c3fded5bb90c0455b0b6dded41658b58ca601ca9da00a6bac3dc85bc'